In [1]:
# https://boysboy3.tistory.com/111?category=765285

In [ ]:
# load package
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys
from matplotlib import pyplot as plt

In [ ]:
# text load 및 load된 text 대문자들을 다 소문자로 변경
filename = "LSTM_1.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [ ]:
# 중복된 문자들을 지우고 정수에 맵핑시키고, 각 단어들을 만든다
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
 
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars) #71469
print ("Total Vocab: ", n_vocab) #68

In [ ]:
# 위에서 만든 dict된 쌍을 input data 및 output(Y) data를 생성한다
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns) #71369
 


In [ ]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# output data를 one-hot-encoding
y = np_utils.to_categorical(dataY)
 
# LSTM model 정의
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
 
 
# 학습단계마다 weights을 저장시키는데 이전보다 결과가 좋아질시 저장
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True,
mode='min')
callbacks_list = [checkpoint]
 
hist = model.fit(X, y, nb_epoch=500, batch_size=128, callbacks=callbacks_list)
 
 
#모델 시각
fig, loss_ax = plt.subplots()
 
acc_ax = loss_ax.twinx()
 
loss_ax.plot(hist.history['loss'], 'y', label='train loss')
 
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
 
loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')
 
plt.show()
 

In [ ]:
# load the network weights
filename = "weights-improvement-374-0.0183.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
 
int_to_char = dict((i, c) for i, c in enumerate(chars))
 
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")